In [1]:
import json

In [2]:
from pyspark.sql import SparkSession

In [3]:
config = json.load(open("config.json"))

In [4]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

24/01/09 00:53:25 WARN Utils: Your hostname, przemyslaw-Latitude-E7450 resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
24/01/09 00:53:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/09 00:53:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 00:53:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/09 00:53:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/01/09 00:53:30 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
dbtable = "equipment"

In [6]:
df_equipment = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [7]:
df_equipment.createOrReplaceTempView(dbtable)

In [8]:
dim_equipment = spark.sql("""
SELECT
ROW_NUMBER() OVER(ORDER BY e.equipment_id) AS equipment_id,
e.equipment_id AS equipment_key,
e.name AS equipment_name,
e.type AS equipment_type,
e.version AS equipment_version
FROM equipment AS e
""")

In [9]:
dim_equipment.show()

24/01/09 00:53:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:53:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:53:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+-------------+----------------+-----------------+-----------------+
|equipment_id|equipment_key|  equipment_name|   equipment_type|equipment_version|
+------------+-------------+----------------+-----------------+-----------------+
|           1|            1|  Cruise control|           Normal|             NULL|
|           2|            2|  Cruise control|           Active|             NULL|
|           3|            3|Air conditioning|            Basic|             NULL|
|           4|            4|Air conditioning|         Two-zone|             NULL|
|           5|            5|Air conditioning|       Multi-zone|             NULL|
|           6|            6|             ABS|           Normal|             NULL|
|           7|            7|            Seat|            Basic|             NULL|
|           8|            8|            Seat|           Heated|             NULL|
|           9|            9|            Seat|Electric Movement|             NULL|
|          10|  

In [11]:
dim_equipment.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "dim_equipment")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()

24/01/09 00:54:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:54:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:54:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:54:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:54:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
